# Introduction

In this project, we will create an artificial neural network that will be trained to predict diseases based on a series of symptoms. In total, there are 132 different symptoms and 42 possible diseases.

The dataset used is not owned by me; all credit for its organization and creation goes to the user kaushil268, who made it freely available on the [Kaggle](https://www.kaggle.com/) website.

You can find everything about the dataset, including the download link, [here](https://www.kaggle.com/datasets/kaushil268/disease-prediction-using-machine-learning).

## About the Dataset

The dataset contains two different files, "Testing" and "Training," both in '.csv' format. We will train our model using the "Training" file and then make predictions on the data in the "Testing" file to verify our model's effectiveness and accuracy.

The "Training" file has 4,921 rows, meaning we have 4,921 different instances for training, and the "Testing" file has 43 instances available for testing our model.